In [1]:
import pandas as pd
import numpy as np
import timeit
import datetime
import urllib.parse
from sqlalchemy import create_engine

In [4]:
# load the agency ratings
ar = AgencyRatings()
ar.load_agency_data(verbose = True)

moodys loaded in 25.8619074 seconds
sp loaded in 21.055627 seconds
fitch loaded in 14.679695299999999 seconds
converting to 8 digit cusip


In [16]:
# get the baml data on a specified date
def get_baml_from_database(date):
    db_connection_string = "Uid=quant;Pwd=r4st3tee;Driver={SQL Server Native Client 11.0};Initial Catalog=Baml;" \
                                           "Server=csnydata01.creditsights.net\csnext;Database=Baml;"

    # Setup database connection
    params = urllib.parse.quote_plus(db_connection_string)
    sqlalchemy = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

    sql = """SELECT *
    FROM dbo.flattened_w_index
    WHERE date = '{}'
    AND (index_name = 'C0A0' OR index_name = 'H0A0')
    """.format(date)

    # read data
    baml = pd.read_sql_query(sql, sqlalchemy)
    
    return baml

start_date = datetime.date(2016,12,31)
end_date = datetime.date(2017,12,31)

# 1. get the starting values (t = 0)

# get the baml constituents on 12/31/2016 (Jan 2017 constituents)
baml = get_baml_from_database('2017-01-03')

# get the 12/31/2016 agency ratings for the constituent set 
baml = ar.get_agency_ratings_by_id(data = baml, id_col = 'cusip', date = start_date)
baml = ar.get_average_ratings(data = baml, 
                              require_two_agencies = False)

baml['mkt_val'] = (baml['price'] / 100) * baml['face_value_loc']
baml['mkt_val'] += (baml['face_value_loc'] / 100) * baml['accrued_interest']

# keep selected columns and rename them
baml_start  = baml[['cusip', 'ticker', 'description', 'ml_industry_lvl_3', 'ml_industry_lvl_4',
                    'average_rating', 'prevmend_oas', 'mkt_val']]
baml_start.rename(columns = {'average_rating': 'average_rating_0',
                            'prevmend_oas': 'oas_0'}, inplace = True)

# 2. get the ending values (t = 1)
# again, get the baml constituents on 12/31/2016 (Jan 2017 constituents)
baml = get_baml_from_database('2017-01-03')
baml = baml[['cusip']]

# now get the spreads of bonds that were in the index a year later on 12/31/2017
baml_end = get_baml_from_database('2017-12-31')
baml_end = baml_end[['cusip', 'oas']]
baml = baml.merge(baml_end, how = 'left', on = 'cusip')


# get the 12/31/2017 agency ratings for the constituent set 
baml_end = ar.get_agency_ratings_by_id(data = baml, id_col = 'cusip', date = end_date)
baml_end = ar.get_average_ratings(data = baml_end, 
                              require_two_agencies = False)
# keep selected columns and rename them
baml_end  = baml_end[['cusip', 'average_rating', 'oas']]
baml_end.rename(columns = {'average_rating': 'average_rating_1',
                            'oas': 'oas_1'}, inplace = True)

# 3. combine the 'start' view with the 'end' view
baml = baml_start.merge(baml_end, how = 'left', on = 'cusip')

# get the change in oas over the period
baml['oas_change'] = baml['oas_1'] - baml['oas_0']
baml.shape




C:\Program Files\Anaconda3\lib\site-packages\pandas\core\frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


(9173, 11)

In [18]:
baml.to_csv('baml_base.csv')

In [17]:
rtm = RatingsTransitionMatrix()
rtm.load_rtm(data = baml)
rtm.load_oas_change_matrix(data = baml)
rtm.get_transition_matrix_3(csv = True)

,Start,Count,AAA,AA1,AA2,AA3,A1,A2,A3,BBB1,...,BB3,B1,B2,B3,CCC1,CCC2,CCC3,CC,C,D
21,AAA,89.0,-19.750322,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20,AA1,56.0,0.000000,-22.831069,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19,AA2,88.0,0.000000,0.000000,-25.923556,-32.593277,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18,AA3,450.0,0.000000,0.000000,-36.904409,-28.821091,-25.371171,-7.612057,-45.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17,A1,649.0,0.000000,0.000000,0.000000,-12.319776,-30.492409,-28.218615,-20.901243,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16,A2,1081.0,0.000000,0.000000,0.000000,0.000000,-24.019827,-27.552365,-38.448717,-87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15,A3,1066.0,0.000000,0.000000,0.000000,0.000000,0.000000,-23.141501,-35.137080,-26.155654,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,BBB1,1345.0,0.000000,0.000000,0.000000,0.000000,0.000000,-49.901220,-41.666789,-33.629949,...,178.453228,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13,BBB2,1277.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-56.192409,-48.643724,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12,BBB3,1012.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-112.000000,-81.736869,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
import collections
import pandas as pd
import numpy as np

class RatingsTransitionMatrix():

    def __init__(self):
        # dictionary from alphanumeric to numeric rating
        self.ratings_map = {'AAA': 21,
                            'AA1': 20, 'AA2': 19, 'AA3': 18,
                            'A1': 17, 'A2': 16, 'A3': 15,
                            'BBB1': 14, 'BBB2': 13, 'BBB3': 12,
                            'BB1': 11, 'BB2': 10, 'BB3': 9,
                            'B1': 8, 'B2': 7, 'B3': 6,
                            'CCC1': 5, 'CCC2': 4, 'CCC3': 3, 'CC': 2, 'C': 1,
                            'D': 0}
        # dictionary from numeric to alphanumeric rating
        self.ratings_map_inverse = {v: k for k,v in self.ratings_map.items()}

        # 1. track the number of issues transitioning from one rating to another
        # dictionary of dictionaries with rating transition **counts**. 
        # Ex: dict['A1']['BBB1'] is the number of cases where ratings went from A1 to BBB1
        self.transition_dict = { r: collections.defaultdict(int) for r in self.ratings_map.keys()}
        
        # 2. track the sum of market value transitioning from one rating to another
        # dictionary of dictionary with rating transitions by market value
        
        # 3. track the weighted average oas 
        self.oas_change_dict = {r: collections.defaultdict(float) for r in self.ratings_map.keys()}
        

        # dictionary with the number of times a bond started with rating X
        self.start_counts = { r: 0.0 for r in self.ratings_map.keys()}

    def load_case(self, start_rating, end_rating):

        # 1. add to the ratings transition matrix
        self.transition_dict[start_rating][end_rating] += 1

        # add to total count of cases that start with a given rating
        self.start_counts[start_rating] += 1
    
    def load_rtm(self, data):
        for i in range(data.shape[0]):
            r1 = data.loc[i, 'average_rating_0']
            r2 = data.loc[i, 'average_rating_1']
            if (r1 != 'NR') and (r2 != 'NR'):
                rtm.load_case(r1, r2)
        return None
    
    def load_oas_change_matrix(self, data):
        
        # make a copy of the data
        temp = data[['cusip', 'mkt_val', 'average_rating_0', 'average_rating_1', 'oas_0', 'oas_1', 'oas_change']].copy()
        mask1 = temp['average_rating_0'] != 'NR'
        mask2 = temp['average_rating_1'] != 'NR'
        mask3 = temp['oas_change'].notnull()
        temp = temp[mask1 & mask2 & mask3]
        
        
        # calc the weighted oas change for each rating transition
        temp['wghtd_oas_change'] = temp['mkt_val'] * temp['oas_change']
        top = temp.groupby(by = ['average_rating_0', 'average_rating_1'])['wghtd_oas_change'].sum()
        bottom = temp.groupby(by = ['average_rating_0', 'average_rating_1'])['mkt_val'].sum()
        wghtd_changes = top / bottom
        
        # convert to dataframe with columns average_rating_0, average_rating_1, wghtd_oas_change
        wghtd_changes = wghtd_changes.to_frame('wghtd_oas_change')
        wghtd_changes.reset_index(inplace = True, drop = False)
        wghtd_changes
        
        # load into a dictionary
        # iterate through each row of the dataframe and load info
        for i in range(wghtd_changes.shape[0]):
            r1 = wghtd_changes.loc[i, 'average_rating_0']
            r2 = wghtd_changes.loc[i, 'average_rating_1']
            val =  wghtd_changes.loc[i, 'wghtd_oas_change']
            if (r1 != 'NR') and (r2 != 'NR'):
                self.oas_change_dict[r1][r2] = val
        return None
        
        

    def get_transition_prob(self, start_rating, end_rating):
        try:
            return self.transition_dict[start_rating][end_rating] / self.start_counts[start_rating]
        except ZeroDivisionError:
            #return 'Error - divide by zero error. There are no cases with a starting rating of {}'.format(start_rating)
            return np.NaN
        except:
            return 'unknown problem'

    def get_upgrade_prob(self, start_rating):
        if self.start_counts[start_rating] == 0:
            return "Sorry, can't calc upgrade prob. No cases with start rating of {}".format(start_rating)
        else:
            numeric_rating = self.ratings_map[start_rating]
            tot_prob = 0.0
            for i in range(numeric_rating +1, 22):
                tot_prob += self.get_transition_prob(start_rating, self.ratings_map_inverse[i])
            return tot_prob

    def get_dwngrade_prob(self, start_rating):
        if self.start_counts[start_rating] == 0:
            return "Sorry, can't calc downgrade prob. No cases with start rating of {}".format(start_rating)
        else:
            numeric_rating = self.ratings_map[start_rating]
            tot_prob = 0.0
            for i in range(numeric_rating):
                tot_prob += self.get_transition_prob(start_rating, self.ratings_map_inverse[i])
            return tot_prob

    def get_default_prob(self, start_rating):
        return self.get_transition_prob(start_rating, 'D')

    def get_expctd_notch_chng(self, start_rating):
        if self.start_counts[start_rating] == 0:
            return "Sorry, can't calc expected notch change. No cases with start rating of {}".format(start_rating)
        else:
            numeric_rating = self.ratings_map[start_rating]
            wghtd_sum = 0.0  # the weighted average notch change

            #iterate over all possible end ratings
            for i in range(0,22):
                end_rating = self.ratings_map_inverse[i] # get the alphanumeric of the end rating
                notch_diff = i - numeric_rating          # get the notches diff between end and start rating
                end_rating_count = self.transition_dict[start_rating][end_rating]  # get the number of times the rating transitioned from start to end
                wght = end_rating_count / self.start_counts[start_rating]
                wghtd_sum += (notch_diff * wght)

            return wghtd_sum

    def get_transition_matrix_1(self, csv = False):
        '''
        transition probabilities
        '''
        df = pd.DataFrame()
        df['Start']  = [self.ratings_map_inverse[x] for x in sorted(self.ratings_map_inverse.keys(), reverse = False) ]
        df['Count'] = [self.start_counts[self.ratings_map_inverse[i]] for i in range(0, 22)]
        for end_rating_numeric in range(21,-1,-1):
            df[self.ratings_map_inverse[end_rating_numeric]] = \
                [self.get_transition_prob(self.ratings_map_inverse[i], self.ratings_map_inverse[end_rating_numeric]) for i in range(0,22)]
        df.sort_index(ascending = False, inplace = True)
        if csv:
            df.to_csv('ratings_transition_matrix.csv')
        return df

    def get_transition_matrix_2(self, csv = False):
        '''
        transitions by bond count
        '''
        df = pd.DataFrame()
        df['Start']  = [self.ratings_map_inverse[x] for x in sorted(self.ratings_map_inverse.keys(), reverse = False) ]
        df['Count'] = [self.start_counts[self.ratings_map_inverse[i]] for i in range(0, 22)]
        for end_rating_numeric in range(21,-1,-1):
            df[self.ratings_map_inverse[end_rating_numeric]] = \
                [self.transition_dict[self.ratings_map_inverse[i]][self.ratings_map_inverse[end_rating_numeric]] for i in range(0,22)]
        df.sort_index(ascending = False, inplace = True)
        if csv:
            df.to_csv('ratings_transition_matrix.csv')
        return df
    
    def get_transition_matrix_3(self, csv = False):
        '''
        weighted-average oas changes
        '''
        df = pd.DataFrame()
        df['Start']  = [self.ratings_map_inverse[x] for x in sorted(self.ratings_map_inverse.keys(), reverse = False) ]
        df['Count'] = [self.start_counts[self.ratings_map_inverse[i]] for i in range(0, 22)]
        for end_rating_numeric in range(21,-1,-1):
            df[self.ratings_map_inverse[end_rating_numeric]] = \
                [self.oas_change_dict[self.ratings_map_inverse[i]][self.ratings_map_inverse[end_rating_numeric]] for i in range(0,22)]
        df.sort_index(ascending = False, inplace = True)
        if csv:
            df.to_csv('ratings_transition_matrix.csv')
        return df




In [2]:

class AgencyRatings():

    '''
    Use this class to access bond-level agency rating data feeds:
    1. get the individual agency ratings
    2. generate agency composite ratings (ACR)

    This class pulls from Y:\QuantitativeStrategy\data-warehouse-exports

    To start, load the agency rating data feeds ingo self.moodys, self.sp, self.fitch by using load_agency_data.
    This is a lot of data! It is therfore slow to load but once it's loaded you have fast access to
    everything as it is stored in memory



    '''

    def __init__(self):
        self.moodys = None
        self.sp = None
        self.fitch = None

        # save baml constituents for use in backfill when we need to search through the baml bonds
        self.baml_constituents = None
        self.baml_constituents_loaded = False

    def load_agency_data(self, verbose = False):
        '''
        get the incremental agency ratings from data warehouse exports
        read in moodys, sp and fitch incremental data and store as attributes of class object
        save in self.moodys, self.sp, self.fitch
        '''

        start = timeit.default_timer()
        moodys = pd.read_csv('Y:\\QuantitativeStrategy\\data-warehouse-exports\\moodys_issue_rating_history.csv')
        #moodys = pd.read_csv('Y:\\QuantitativeStrategy\\staging-dw-exports\\moodys_issue_rating_history.csv')
        #moodys = pd.read_csv('moodys_issue_rating_history.csv')

        if verbose:
            print('moodys loaded in {} seconds'.format(timeit.default_timer() - start))

        start = timeit.default_timer()
        sp = pd.read_csv('Y:\\QuantitativeStrategy\\data-warehouse-exports\\s_p_issue_rating_history.csv')
        #sp = pd.read_csv('Y:\\QuantitativeStrategy\\staging-dw-exports\\s_p_issue_rating_history.csv')

        if verbose:
            print('sp loaded in {} seconds'.format(timeit.default_timer() - start))

        # get the csv files
        start = timeit.default_timer()
        fitch = pd.read_csv('Y:\\QuantitativeStrategy\\data-warehouse-exports\\fitch_issue_rating_history.csv')
        #fitch = pd.read_csv('Y:\\QuantitativeStrategy\\staging-dw-exports\\fitch_issue_rating_history.csv')
        #fitch = pd.read_csv('fitch_issue_rating_history.csv')

        if verbose:
            print('fitch loaded in {} seconds'.format(timeit.default_timer() - start))

        # the agency rating feed gives cusip as 9 digits
        # but many sources like baml might only give 8 cusips (no check digit)
        # for consistency, convert all cusips in the incremental agency rating data to 8 digits
        # (for isins, assume 12 digits)
        if verbose:
            print('converting to 8 digit cusip')

        mask = sp['id_type'].isin(['Cusip1', 'Cusip2', 'Cusip3', 'Cusip4', 'Cusip5', 'Cusip6'])
        sp.loc[mask, 'id_value'] = sp.loc[mask, 'id_value'].map(lambda x: x[:8])

        mask = fitch['id_type'].isin(['Cusip1', 'Cusip2', 'Cusip3', 'Cusip4', 'Cusip5', 'Cusip6'])
        fitch.loc[mask, 'id_value'] = fitch.loc[mask, 'id_value'].map(lambda x: x[:8])

        mask = moodys['id_type_text'].isin(['CUSIP',
                                            'CUSIP 3',
                                            'CUSIP 4',
                                            'CUSIP 5',
                                            'CUSIP - Previous',
                                            'CUSIP - Second',
                                            'CUSIP-2ndary Wrap Orig. CUSIP',
                                            'CUSIP-Deriv/Underlying Bond'])
        moodys.loc[mask, 'instrument_id_value'] = moodys.loc[mask, 'instrument_id_value'].map(lambda x: x[:8])

        # for moodys, only keep certain types of ratings
        # exclude ratings like bank credit facility, preferred stock
        # sometimes a bond can have multiples types of ratings, but we only want the 'regular bond rating'
        mask1 = moodys['security_class_short_description'] == 'REG'  # regular bond/debenture
        mask2 = moodys['security_class_short_description'] == 'MTN'  # medium term note
        mask3 = moodys['security_class_short_description'] == 'PRF'  # medium term note
        mask4 = moodys['security_class_short_description'] == 'CON'  # medium term note
        moodys = moodys[mask1 | mask2 | mask3 | mask4]

        # exclude LGD ratings
        mask = moodys['rating_class_text'].map(lambda x: 'LGD' in x)
        moodys = moodys[-mask]

        # save as attributes of class
        self.moodys = moodys
        self.sp = sp
        self.fitch = fitch

    def get_fitch_ratings(self, data, id_col, date = 'current'):

        '''
        attach a column with fitch ratings to a dataset

        pass in a dataframe with a group of bonds in the rows. we want to add a new column with the fitch rating

        :param data: a dataset that contains bonds that you want the rating for, as dataframe
        :id_col: the name of the column in the datset that contains either the cusip or isin, as string
        :param date: date(s) of the ratings you want, either 'current', a date in datetime.date, or 'incremental'
        :return: a dataset with an added fitch_rating column, as dataframe
        '''

        original = data.copy()

        # organize the bonds you want to get ratings for
        # keep just a dataframe with a single column of bond identifiers (cusip or isin)
        df = data.copy()
        df = df[[id_col]]

        # merge the entire incremental fitch ratings dataset and only keep bonds from the target group above
        df = df.merge(self.fitch, how = 'left', left_on = id_col, right_on = 'id_value')


        # data cleanup
        # set a datetime object and sort
        df['long_term_issue_rating_effective_date'] = pd.to_datetime(df['long_term_issue_rating_effective_date'])
        df.sort_values(by = [id_col, 'long_term_issue_rating_effective_date'], inplace = True)

        # get the ratings you want
        # if you just want the current ratings, then keep the last rating action for each bond
        if date == 'current':
            df.drop_duplicates(subset = id_col, keep = 'last', inplace = True)
        # but if you want a record of all ratings actions, don't drop anything
        elif date == 'incremental':
            pass
        # and if you want a rating on a specific historical date,
        # then keep the most recent rating prior to the historical date
        else:
            end_of_date = datetime.datetime(date.year, date.month, date.day, 23,59,59)
            df = df[df['long_term_issue_rating_effective_date'] <= end_of_date]
            df.drop_duplicates(subset = id_col, keep = 'last', inplace = True)

        # data cleanup
        fitch_fields = ['agent_common_id',
                        'issuer_name',
                        'fitch_issue_id_number',
                        'id_type',
                        'id_value',
                        'issue_description',
                        'long_term_issue_rating_effective_date']

        # delete columns that aren't needed
        for f in fitch_fields:

            # if incremental production, then keep everything
            if (date == 'incremental') & (f == 'long_term_issue_rating_effective_date'):
                pass
            # but if current or historical production, just keep the rating, rating date, and seniority
            else:
                del df[f]

        # data cleanup
        df.rename(columns = {'long_term_issue_rating': 'fitch_rating',
                            'long_term_issue_rating_effective_date': 'rating_date',
                            'issue_debt_level_code': 'fitch_seniority'}, inplace = True)

        return df

    def get_moodys_ratings(self, data, id_col, date):
        '''
        attach a column with moodys ratings to a dataset
        :param data: a dataset that contains bonds that you want the rating for, as dataframe
        :id_col: the name of the column in the datset that contains either the cusip or isin, as string
        :param date: date(s) of the ratings you want, either 'current', a date in datetime.date format, or 'incremental'
        :return: a dataset with an added moodys_rating column, as dataframe
        '''


        df = data.copy()
        df = df[[id_col]]


        df = df.merge(self.moodys, how = 'left', left_on = id_col, right_on = 'instrument_id_value')

        df['rating_date'] = pd.to_datetime(df['rating_date'])
        df.sort_values(by = [id_col, 'rating_date'], inplace = True)

        if date == 'current':
            df.drop_duplicates(subset = id_col, keep = 'last', inplace = True)
        elif date == 'incremental':
            pass
        else:
            end_of_date = datetime.datetime(date.year, date.month, date.day, 23,59,59)
            df = df[df['rating_date'] <= end_of_date]
            df.drop_duplicates(subset = id_col, keep = 'last', inplace = True)

        moodys_fields = ['instrument_id',
                         'moodys_rating_id',
                         'security_class_short_description',
                         'id_type_text',
                         'instrument_id_value',
                         'rating_date',
                         'rating_class_text',
                         'rating_direction_short_description',
                         'rating_type_short_description',
                         'rating_currency_iso_code']
        for f in moodys_fields:
            if (date == 'incremental') & (f == 'rating_date'):
                pass
            else:
                del df[f]
        df.rename(columns = {'rating_text': 'moodys_rating',
                            'seniority_short_description': 'moodys_seniority'}, inplace = True)

        return df

    def get_sp_ratings(self, data, id_col, date):
        '''
        attach a column with S&P ratings to a dataset
        :param data: a dataset that contains bonds that you want the rating for, as dataframe
        :id_col: the name of the column in the datset that contains either the cusip or isin, as string
        :param date: date(s) of the ratings you want, either 'current', a date in datetime.date format, or 'incremental'
        :return: a dataset with an added sp_rating column, as dataframe
        '''

        df = data.copy()
        df = df[[id_col]]

        df = df.merge(self.sp, how = 'left', left_on = id_col, right_on = 'id_value')

        df['rating_date'] = pd.to_datetime(df['rating_date'])
        df.sort_values(by = [id_col, 'rating_date'], inplace = True)

        if date == 'current':
            df.drop_duplicates(subset = id_col, keep = 'last', inplace = True)
        elif date == 'incremental':
            pass
        else:
            end_of_date = datetime.datetime(date.year, date.month, date.day, 23,59,59)
            df = df[df['rating_date'] <= end_of_date]
            df.drop_duplicates(subset = id_col, keep = 'last', inplace = True)

        sp_fields = ['security_id',
                     'security_symbol_value',
                     'id_type',
                     'id_value',
                     'rating_date']

        for f in sp_fields:
            # don't delete rating date if incremental
            if (date == 'incremental') & (f == 'rating_date'):
                pass
            else:
                del df[f]
        df.rename(columns = {'rating': 'sp_rating'}, inplace = True)

        return df

    def get_time_series_by_id(self, data, id_col, start_date, end_date, verbose = False):
        '''
        generate a daily time series of ratings given a set of bonds
        :param data: a dataset that contains bonds that you want the rating for, as dataframe
        :id_col: the name of the column in the datset that contains either the cusip or isin, as string
        :param start_date: start date of the time series in 'YYYY-MM-DD' format
        :param end_date: end date of the time series in 'YYYY-MM-DD' format
        :return: a time series dataset with an added moodys_rating, sp_rating, fitch_rating columns, as dataframe
        '''

        # get the incremental ratings from each agency for the given set of bonds
        if verbose:
            print('get incremental ratings for given bonds')
        moodys = self.get_moodys_ratings(data, id_col, date = 'incremental')
        sp = self.get_sp_ratings(data, id_col, date = 'incremental')
        fitch = self.get_fitch_ratings(data, id_col, date = 'incremental')

        if verbose:
            print('--process incremental ratings for combination with date template')
        for db in [moodys, sp , fitch]:

            # sort by date-time
            db.sort_values(by = [id_col, 'rating_date'], inplace = True)

            # convert date-time to just dates
            # (otherwise cannot properly merge into the date range below which are date types, not date-time types)
            db['rating_date'] = db['rating_date'].dt.date

            # keep the last action when multiple rating actions on the same date
            db.drop_duplicates(subset = [id_col, 'rating_date'], keep = 'last', inplace = True)

        # convert incremental data to daily time series
        # generate a series with daily dates
        if verbose:
            print('--create a date template')
        dates = pd.date_range(start = start_date, end = end_date, freq = 'D')
        dates = pd.DataFrame(dates)
        dates.rename(columns = {0: 'date'}, inplace = True)
        dates['date'] = dates['date'].dt.date    # convert datetime to just date
        dates['join'] = 1
        dates['from_date_template'] = 1

        # get a df with a columns of all cusips/isins
        bonds = data.copy()
        bonds = bonds[[id_col]]
        bonds.drop_duplicates(subset = id_col, inplace = True)
        bonds['join'] = 1

        # combine daily date range with the bonds
        # this will produce a dataframe with a daily observation for every bond
        # two columns: date, bond with rows:
        # bond1-day1
        # bond1-day2
        # ...
        # bond1-dayN
        # bond2-day1
        # bond2-day2
        # ...
        # bond2-dayN
        dates = dates.merge(bonds, how = 'left', left_on = 'join', right_on = 'join')
        del dates['join']

        # merge in the incremental moodys ratings to the date template
        dates = dates.merge(moodys, how = 'outer', left_on = [id_col, 'date'], right_on=[id_col, 'rating_date'])

        # find places where we have a rating_date but no date
        # ie the date_range was from 2000-2017 but the first rating date was 1995
        mask1 = (dates['date'].isnull()) & (dates['rating_date'].notnull())

        # set date to rating_date in the above cases
        dates.loc[mask1, 'date'] = dates.loc[mask1, 'rating_date']
        del dates['rating_date']

        # repeat for s&p ratings
        dates = dates.merge(sp, how = 'outer', left_on = [id_col, 'date'], right_on=[id_col, 'rating_date'])
        mask1 = (dates['date'].isnull()) & (dates['rating_date'].notnull())
        dates.loc[mask1, 'date'] = dates.loc[mask1, 'rating_date']
        del dates['rating_date']

        # repeat for fitch ratings
        dates = dates.merge(fitch, how = 'outer', left_on = [id_col, 'date'], right_on=[id_col, 'rating_date'])
        mask1 = (dates['date'].isnull()) & (dates['rating_date'].notnull())
        dates.loc[mask1, 'date'] = dates.loc[mask1, 'rating_date']
        del dates['rating_date']

        dates.loc[dates['from_date_template'].isnull(), 'from_date_template'] = 0

        dates.sort_values(by = [id_col, 'date'], inplace = True)

        # fill forward the ratings to convert from incremental to daily
        dates = dates.groupby(by = id_col, as_index = False).fillna(method='ffill')

        # drop cases that are not from the date template
        # ie cases where we instantiate the rating at 1/1/1900
        dates = dates[dates['from_date_template'] == 1]
        del dates['from_date_template']
        return dates



    def get_agency_ratings_by_id(self, data, id_col, date = 'current'):
        '''
        pass in a dataset that contains a column with cusips that you want to get agency ratings for
        get the agency rating for either the 'current' date or a specified historical date
        :param data: a dataset that contains bonds that you want the rating for, as dataframe
        :id_col: the name of the column in the datset that contains either the cusip or isin, as string
        :param date: date(s) of the ratings you want, either 'current', a date in datetime.date format, or 'incremental'
        :return: a dataset with an added moodys_rating column, as dataframe

        '''

        assert date != 'incremental', 'error: cannot use incremental ratings'
        assert id_col in data.columns, 'error: could not find the id column in data'
        if date != 'current':
            assert isinstance(date, datetime.date), 'error: for non current date values you must pass date as datetime.date'

        # get fitch ratings
        fitch = self.get_fitch_ratings(data, id_col, date)
        moodys = self.get_moodys_ratings(data, id_col, date)
        sp = self.get_sp_ratings(data, id_col, date)

        df = data.copy()

        df = df.merge(moodys, how = 'left', left_on = id_col, right_on = id_col)

        df = df.merge(sp, how = 'left', left_on = id_col, right_on = id_col)
        df = df.merge(fitch, how = 'left', left_on = id_col, right_on = id_col)

        for rating in ['moodys_rating', 'sp_rating', 'fitch_rating']:
            df.loc[df[rating].isnull(), rating] = 'NR'

        return df

    def get_average_ratings(self, data, require_two_agencies = True):
        '''
        calculate the average agency rating
        :param data: , a dataset with columns for moodys, sp and fitch alphanumeric ratings, as dataframe
        :param require_two_agencies: require at least two agency ratings in order to calculate average, as boolean,
        :return: the input dataset with new columns for average ratings
        '''

        for c in ['moodys_rating', 'sp_rating', 'fitch_rating']:
            assert c in data.columns, 'error: cannot find {} in data'.format(c)

        # mapping from alphanumeric to numeric rating
        self.numeric_dict = {'AAA': 21,
                            'AA1': 20, 'AA2': 19, 'AA3': 18,
                            'A1': 17, 'A2': 16, 'A3': 15,
                            'BBB1': 14, 'BBB2': 13, 'BBB3': 12,
                            'BB1': 11, 'BB2': 10, 'BB3': 9,
                            'B1': 8, 'B2': 7, 'B3': 6,
                            'CCC1': 5, 'CCC2': 4, 'CCC3': 3,
                            'CC': 2, 'C': 1, 'D': 0,

                            'Aaa': 21, 'Aa1': 20, 'Aa2': 19, 'Aa3': 18,
                            'A1': 17, 'A2': 16, 'A3': 15,
                            'Baa1': 14, 'Baa2': 13, 'Baa3': 12,
                            'Ba1': 11, 'Ba2': 10, 'Ba3': 9,
                            'Caa1': 5, 'Caa2': 4, 'Caa3': 3,
                            'Ca': 2, 'C': 1,

                            'AA+': 20, 'AA': 19, 'AA-': 18,
                            'A+': 17, 'A': 16, 'A-': 15,
                            'BBB+': 14, 'BBB': 13, 'BBB-': 12,
                            'BB+': 11, 'BB':10, 'BB-': 9,
                            'B+': 8, 'B':7, 'B-': 6,
                            'CCC+': 5, 'CCC': 4, 'CCC-': 3,

                            'SD': 0, 'RD': 0, 'WR': np.NaN, 'NR': np.NaN, 'WD': np.NaN
                             }

        # mapping from numeric to alphanumeric rating
        self.alphanumeric_dict = {21: 'AAA',
                                  20: 'AA1', 19: 'AA2', 18: 'AA3',
                                  17: 'A1', 16: 'A2', 15: 'A3',
                                  14: 'BBB1', 13: 'BBB2', 12: 'BBB3',
                                  11: 'BB1', 10: 'BB2', 9: 'BB3',
                                  8: 'B1', 7: 'B2', 6: 'B3',
                                  5: 'CCC1', 4: 'CCC2', 3: 'CCC3',
                                  2: 'CC', 1: 'C', 0: 'D',
                                  'NaN': 'NR'
                                  }

        df = data.copy()

        # map alphanumeric ratings to a number
        df['moodys_num'] = df['moodys_rating'].map(self.numeric_dict)
        df['sp_num'] = df['sp_rating'].map(self.numeric_dict)
        df['fitch_num'] = df['fitch_rating'].map(self.numeric_dict)

        # calculate average agency rating
        # offset numeric average by a small amount so that X.5 it gets rounded down to X and not rounded up to X + 1
        df['average_rating_num'] = df[ ['moodys_num', 'sp_num', 'fitch_num'] ].apply(np.mean, axis = 1) - 0.0002
        mask = df['average_rating_num'].notnull()
        df.loc[mask, 'average_rating_num'] = df.loc[mask, 'average_rating_num'].map(round)
        df['agency_rating_count'] = df[ ['moodys_num', 'sp_num', 'fitch_num'] ].count(axis = 1)

        del df['moodys_num']
        del df['sp_num']
        del df['fitch_num']

        # null average if less than two agency ratings
        if require_two_agencies == True:
            mask1 = df['agency_rating_count'] < 2
            df.loc[mask1, 'average_rating_num'] = np.NaN

        # notching based on seniority
        # TO DO

        # map numeric average to alphanumeric rating
        df['average_rating'] = df['average_rating_num'].map(self.alphanumeric_dict)
        df['average_rating'] = df['average_rating'].fillna('NR')

        return df